In [ ]:
# Import required library
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

In [ ]:
#upload file in colab
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv


In [ ]:
#loading rating and movie data
rate = pd.read_csv('ratings.csv')
movie = pd.read_csv('movies.csv')

In [ ]:
#merge two csv data into one dataframe
newdf = pd.merge(rate, movie, on= 'movieId').drop(['timestamp', 'genres'], axis =1)
newdf.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [ ]:
#Pivot so that dataset get summarriezed based on userId as row and movie as column
pivoting = newdf.pivot_table(index='userId',columns='title',values='rating')
pivoting.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Checking for how many movie single user has rated
rate = {}
rows_indexes = {}
for i, row in pivoting.iterrows():
  rows = [x for x in range(0, len(pivoting.columns))]
  combine = list(zip(row.index, row.values, rows))
  rated = [(x,z) for x,y,z in combine if str(y) != 'nan']
  index = [i[1] for i in rated]
  row_names = [i[0] for i in rated]
  rows_indexes[i] = index
  rate[i] = row_names

In [ ]:
rate[400]

['Arrival (2016)',
 'Back to the Future (1985)',
 'Blade Runner (1982)',
 'Dark Knight, The (2008)',
 'Departed, The (2006)',
 'Die Hard (1988)',
 'Donnie Darko (2001)',
 'Fargo (1996)',
 'Fight Club (1999)',
 'Forrest Gump (1994)',
 'Gladiator (2000)',
 'Godfather, The (1972)',
 'Godfather: Part II, The (1974)',
 'Goodfellas (1990)',
 'Heat (1995)',
 'How to Train Your Dragon (2010)',
 'Inception (2010)',
 'Indiana Jones and the Kingdom of the Crystal Skull (2008)',
 'Inside Man (2006)',
 'Logan (2017)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lucky Number Slevin (2006)',
 'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
 'Matrix, The (1999)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Princess Bride, The (1987)',
 'Pulp Fiction (1994)',
 'Pursuit of Happyness, The (2006)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Requiem for a Dream (2000)',

In [ ]:
rate

user-item matrix showing how much rate the user have provided to item they have engaged in.

In [ ]:
#Filling the nan value with 0. But this doesn't mean user disliked movie
rpivoting = newdf.pivot_table(index='userId',columns='title',values='rating').fillna(0)
rpivoting = rpivoting.apply(np.sign)
rpivoting.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


not rated movie by user

In [ ]:
#Movie not rated by the user. This list which movie are not rated by a particular user. Useful when doing recommendation
norate = {}
norate_indexes = {}
for i, row in rpivoting.iterrows():
  rows = [x for x in range(0, len(pivoting.columns))]
  combine = list(zip(row.index, row.values, rows))
  idx_row = [(idx,col) for idx, val, col in combine if not val>0]
  indices = [i[1] for i in idx_row]
  row_names = [i[0] for i in idx_row]
  norate_indexes[i] = indices
  norate[i] = row_names

In [ ]:
norate[400]

Unsupervised Nearest neighbor Recommender

In [ ]:
# K nearest neighbor is formulated to model for unsurpervised learning. 
# for the purpose of cross validation 5 fold is choosen. 
#Metrics to find similarity used is cosine

n=5             #5 fold cross-validation

#CHOSEN METHOD COSINE SIMILARITY
#cosine similarity calculation
cosine_knn = NearestNeighbors(n_neighbors= n, algorithm= 'brute', metric= 'cosine')
item_knn_fit = cosine_knn.fit(rpivoting.T.values)

#counterpart method
#pearson correlation calculation
# pearson_knn = NearestNeighbors(n_neighbors= n, algorithm= 'brute', metric= 'correlation')
# item_knn_fit = pearson_knn.fit(rpivoting.T.values)

i_dist, i_indices = item_knn_fit.kneighbors(rpivoting.T.values)

#----comment and uncomment appropriate section for implementing different relation modeling approach.------

item-based recommend

In [ ]:
#this section build of item list that to be suggested
items_dict = {}
for i in range(len(rpivoting.T.index)):
  item_index = i_indices[i]
  col_names = rpivoting.T.index[item_index].tolist()
  items_dict[rpivoting.T.index[i]] = col_names

In [ ]:
#Based on given user and item the distance is calculated to know similar user and made the classification for each individual where they fall. 
#Then list of movie is created to recommend to particular group
recon = {}
for u,v in rows_indexes.items():
  item_idx = [j for i in i_indices[v] for j in i]
  item_dist = [j for i in i_dist[v] for j in i]
  combine = list(zip(item_dist, item_idx))
  disn = {i:d for d,i in combine if i not in v}
  zipped = list(zip(disn.keys(),disn.values()))
  sort = sorted(zipped, key= lambda x: x[1])
  recomm = [(rpivoting.columns[i], d) for i,d in sort]
  recon[u] = recomm

In [ ]:
#Function to handle the recommendation for given user present in the given list.
#nor here mean how many movie to recommend to given user. Here we are recommending 5 movies at one time for given user
def dorecom(user, nor = 5):
  if user > len(rpivoting.index):
    print('no user exists')
  else:
    print("recommended movies:\n")
    for u,v in recon.items():
      if user == u:
        for i in v[:nor]:
          print('{} with similarity: {:.4f}'.format(i[0],1-i[1]))


In [ ]:
#Function call for user. Pass user index for whom you want recommendation be generated
dorecom(400)

recommended movies:

Lord of the Rings: The Two Towers, The (2002) with similarity: 0.8633
Jurassic Park (1993) with similarity: 0.7076
Pirates of the Caribbean: The Curse of the Black Pearl (2003) with similarity: 0.7047
Iron Man (2008) with similarity: 0.6929
Saving Private Ryan (1998) with similarity: 0.6824


Output above shows the movie which are similar and nearest in cluster based in knn lazy learning (use of cosine distance) . And the value depicts how close the distance between item is present.

Rating prediction for new movie which user has not seen

In [ ]:
#prediction of rating of movie for given user
i_dist = 1-i_dist
prediction = i_dist.T.dot(rpivoting.T.values)/np.array([np.abs(i_dist.T).sum(axis=1)]).T
groundTruth = rpivoting.T.values[i_dist.argsort()[0]]

Performance evaluation of recommending system

In [ ]:
def rmse(prediction, groundTruth):
  prediction = prediction[groundTruth.nonzero()].flatten()
  groundTruth = groundTruth[groundTruth.nonzero()].flatten()
  return sqrt(mean_squared_error(prediction, groundTruth))

In [ ]:
#making prediction and compare with actual rate given to movie
#here Root mean squared is used to test the model performance
error_rate = rmse(prediction, groundTruth)
print("Accuracy : {:.3f}".format(100-error_rate))
print("RMSE : {:.5f}".format(error_rate))

Accuracy : 99.049
RMSE : 0.95084
